# Match further climate change contrarians

#### This script matches climate change contrarians according to Farell's list of denialists organisations and drops a couple of hearings that should not be included in the analysis

In [1]:
from TextCollection import *
import keyboard  # using module keyboard

In [2]:
# # Reinstantiate class after changing the Textcollection.py script
# # Do not run this in the last run! Leads to a conflict with pickle.

# from importlib import reload

# os.chdir('/home/mirjam/OneDrive/congress_committees/ArticleOne/Article_Scripts/Hearings')
# import TextCollection; reload(TextCollection)

# # Reinstantiate class
# t.__class__ = HearingsCollection

In [3]:
# Change directory
os.chdir('../../Data/')

In [4]:
# Run for maximum recursion error during saving

import resource
import sys

print(resource.getrlimit(resource.RLIMIT_STACK))
print(sys.getrecursionlimit())

# May segfault without this line. 0x100 is a guess at the size of each stack frame.
max_rec = 0x100000
resource.setrlimit(resource.RLIMIT_STACK, [0x100 * max_rec, resource.RLIM_INFINITY])
sys.setrecursionlimit(max_rec)

print(sys.getrecursionlimit())

(8388608, -1)
3000
1048576


<br>

## 1) Preparation: Loading, correcting and inspecting the data

In [5]:
t = load('Hearings/05_witnesses.pkl')
with open('OpenSecrets/sectors_industries_contributors.json', 'r') as jfile:
    sectors = json.load(jfile)
print('We imported {} hearings and lobbying {} sectors for the years {} to {}.\n'.format(len(t),len(sectors), sectors[0]['industries'][0]['year'][0], 
                                                                                                        sectors[0]['industries'][0]['year'][-1]))
# Append sectors to hearings collection
t.sectors = sectors

We imported 263 hearings and lobbying 13 sectors for the years 2003 to 2010.



In [6]:
t.texts[0].keys()

dict_keys(['filename', 'identifier', 'content_raw', 'content', 'content_stripped', 'congress', 'climate_related', 'keywords_climate', 'keywords_carbonpricing', 'keywords_climatechange', 'title', 'date', 'year', 'committee', 'MODS', 'witnesses', 'desmog_witness', 'desmog_organisation', 'witness_affiliation', 'witness_sector', 'witness_industry', 'state_research'])

<br>

## 2)  Find more climate change contrarians

### Find more climate change contrarians 

#### Scan for contrarians on Farell's list of denialists organisations 
(https://media.nature.com/original/nature-assets/nclimate/journal/v6/n4/extref/nclimate2875-s1.pdf) 
#### Scan for the 'National Petrochemical & Refiners Association', which is the former name of the 'American Fuel & Petrochemical Manufacturers' 
(https://www.desmogblog.com/american-fuel-petrochemical-manufacturers-afpm)

In [7]:
contrarian_organisations = ['60 plus association', 'accuracy in media', 'action institute for the study of religion and liberty', 
                           'advancement of sound science center inc', 'alexis de tocqueville institution', 'american coal foundation', 
                           'american coalition for clean coal electricity', 'american conservative union foundation', 
                           'american council for capital formation', 'american council on science and health', 'american energy alliance', 
                           'american energy freedom center', 'american enterprise institute for public policy research', 
                           'american farm bureau federation', 'american friends of institute of economic affairs', 
                           'american fuel and petrochemical manufacturers', 'american gas association', 'american legislative exchange council', 
                           'american natural gas alliance inc', 'american petroleum institute', 'american policy center', 
                           'american spectator foundation', 'american tradition institute', 'americans for a limited government inc', 
                           'americans for balanced energy choices', 'americans for prosperity', 'americans for tax reform', 
                           'annapolis center for science based public policy inc', 'association of global automobile manufacturers inc', 
                           'atlantic legal foundation', 'atlas economic research foundation atlas', 'australian climate science coalition', 
                           'capital research center and greenwatch', 'cascade policy institute', 'cato institute', 
                           'center for american and international law', 'center for defense of free enterprise', 
                           'center for security policy inc', 'center for strategic and international studies', 
                           'center for study of carbon dioxide and global change', 'centre for new europe', 
                           'chamber of commerce of united states of america', 'charles koch institute', 
                           'citizens for a sound economy now freedomworks', 'citizens for affordable energy inc', 'co2 is green inc', 
                           'coalition for american jobs', 'coalition for vehicle choice inc', 'collegians for constructive tomorrow', 
                           'committee for constructive tomorrow', 'communications institute', 'competitive enterprise institute', 
                           'consumer alert inc', 'consumer energy alliance inc', 'consumers alliance for global prosperity', 
                           'cooler heads coalition', 'cornwall alliance for the stewardship of creation', 'dci group', 
                           'defenders of property rights', 'donors trust donors capital fund', 'edison electric institute', 
                           'energy makes america great', 'environmental conservation organization', 'environmental literacy council', 
                           'exxonmobil', 'federalist society for law and public policy studies', 
                           'federation for american coal energy and security', 'fraser institute', 
                           'free enterprise action institute free enterprise education institute', 'freedom action', 'freedomworks foundation', 
                           'freedomworks inc', 'frontiers of freedom institute inc', 'george marshall institute', 
                           'george mason university law and economics center gmu lec', 'global climate coalition', 
                           'global warming policy foundation', 'greening earth society', 'heartland institute', 'heritage foundation', 
                           'hoover institution on war revolution and peace stanford university', 'hudson institute', 
                           'illinois policy institute', 'independence institute', 'independent commission on environmental education', 
                           'independent institute', 'independent petroleum association of america', 'independent womens forum', 
                           'industrial energy consumers of america', 'initiative for public policy analysis', 
                           'institute for biospheric research', 'institute for energy research', 'institute for liberty', 
                           'institute for regulatory science', 'institute for study of earth and man', 
                           'institute of humane studies george mason university', 'institute of public affairs', 
                           'intermountain rural electric association', 'international climate and environmental change assessment project', 
                           'international climate science coalition', 'international council for capital formation', 
                           'international policy network', 'international republican institute iri', 
                           'james madison institute for public policy studies inc', 'john locke foundation inc', 'junkscience dot com', 
                           'knowledge and progress fund', 'koch foundations combined', 'koch industries', 'landmark legal foundation', 
                           'lexington institute', 'lindenwood university', 'locke institute', 'mackinac center', 
                           'manhattan institute for policy research inc', 'media research center inc', 'mercatus center inc gwu', 
                           'mountain states legal foundation', 'national association of manufacturers of usa', 
                           'national black chamber of commerce', 'national center for policy analysis', 
                           'national center for public policy research inc', 'national council for environmental balance', 
                           'national environmental policy institute', 'national legal center for public interest', 
                           'national mining association', 'national petroleum council', 'national policy forum', 
                           'national rural electric cooperative association', 'national taxpayers union', 
                           'national taxpayers union foundation', 'national wilderness institute', 'new zealand climate science coalition', 
                           'oklahoma council of public affairs inc', 'oregon institute of science and medicine', 'pacific legal foundation', 
                           'pacific research institute for public policy', 'peabody energy', 'plants need co2 org', 
                           'property and environment research center', 'reason foundation', 'responsible resources', 
                           'science and environmental policy project', 'science and public policy institute', 'shook hardy and bacon llp', 
                           'small business survival committee', 'smithsonian astrophysical observatory willie soon and sallie baliunas', 
                           'southeastern legal foundation inc', 'sovereignty international inc', 'state policy network', 
                           'statistical assessment service stats', 'tech central science foundation', 'texas public policy foundation', 
                           'thomas jefferson institute for public policy', 'ts august', 'united for jobs', 'us russia business council', 
                           'virginia institute for public policy', 'washington legal foundation', 'washington policy center', 
                           'weidenbaum center on the economy government and public policy', 'center for the study of american business', 
                           'western fuels association', 'world affairs councils of america', 'world climate report',
                           'national petrochemical and refiners association']

In [8]:
count = 0
for i, text in enumerate(t.texts):
    t.texts[i]['contrarian_organisation'] = t.texts[i]['desmog_organisation'].copy()
    for j, contrarian_organisation in enumerate(text['contrarian_organisation']):
        if contrarian_organisation is None:
            for organisation in contrarian_organisations:
                ratio = fuzz.partial_ratio(organisation,text['witnesses'][j].lower())
                if ratio >= 90:
# Uncomment the following lines to see the matched contrarian witnesses
#                     print(i, j, ratio, ': ',
#                           text['witnesses'][j].lower(),'\n\t',
#                           organisation, '\n')
                    count+=1
                    t.texts[i]['contrarian_organisation'][j] = organisation
                    break
print('{} further contrarian witnesses were matched.'.format(count))

27 further contrarian witnesses were matched.


<br>

## 4)  Extract the correct committee data from the MOD files for each hearing

In [9]:
for i, text in enumerate(t.texts):
    if len(re.findall('<name type="authority-standard">\W?(.+?)\W?</name>', str(text['MODS']))) != 0:
        print(i, text['identifier'], re.findall('<name type="authority-standard">\W?(.+?)\W?</name>', str(text['MODS'])))
        t.texts[i]['committee'] =re.findall('<name type="authority-standard">\W?(.+?)\W?</name>', str(text['MODS']))
    else:
        print(i, text['identifier'], re.findall('<otherhostingorg>(.+?)(?: House of Representatives)?</otherhostingorg>', str(text['MODS'])))
        t.texts[i]['committee'] = re.findall('<otherhostingorg>(.+?)(?: House of Representatives)?</otherhostingorg>', str(text['MODS']))

0 111hhrg67814 ['Committee on Energy and Commerce']
1 110hhrg62523 ['Permanent Select Committee on Intelligence']
2 110hhrg58149 ['Select Committee on Energy Independence and Global Warming']
3 108hhrg90674 ['Committee on Science']
4 110hhrg35774 ['Committee on Ways and Means']
5 110hhrg41132 ['Committee on Science and Technology']
6 110hhrg61959 ['Select Committee on Energy Independence and Global Warming']
7 110hhrg48374 ['Committee on Agriculture']
8 111shrg52750 ['Committee on Commerce, Science, and Transportation']
9 110hhrg62201 ['Committee on Ways and Means']
10 110shrg36640 ['Committee on Energy and Natural Resources']
11 111hhrg62591 ['Select Committee on Energy Independence and Global Warming']
12 110hhrg36921 ['Committee on Energy and Commerce']
13 110shrg35525 ['Committee on Homeland Security and Governmental Affairs']
14 110hhrg42305 ['Committee on Transportation and Infrastructure']
15 111hhrg73749 ['Committee on Energy and Commerce']
16 111hhrg62186 ['Select Committee on

135 111hhrg52329 ['Committee on Agriculture']
136 111hhrg48662 ['Committee on Natural Resources']
137 110hhrg50023 ['Committee on Energy and Commerce']
138 110shrg44855 ['Committee on Energy and Natural Resources']
139 110shrg80088 ['Committee on Commerce, Science, and Transportation']
140 111shrg50178 ['Committee on Energy and Natural Resources']
141 111shrg47252 ['Committee on Energy and Natural Resources']
142 111hhrg51519 ['Committee on Small Business']
143 111shrg55049 ['Committee on Energy and Natural Resources']
144 111hhrg57172 ['Committee on Science and Technology']
145 108shrg94602 ['Committee on Environment and Public Works']
146 111hhrg51949 ['Committee on Ways and Means']
147 110shrg36310 ['Committee on Energy and Natural Resources']
148 110hhrg62525 ['Select Committee on Energy Independence and Global Warming']
149 110shrg34079 ['Committee on Energy and Natural Resources']
150 111hhrg73745 ['Committee on Energy and Commerce']
151 111hhrg50438 ['Committee on Natural Resour

In [10]:
# Check the hearings that have not been assigned exactly one committee
for i, text in enumerate(t.texts):
    if len(text["committee"]) != 1:
        print(i, text['identifier'], text['committee'])

25 108shrg91254 ['Committee on Appropriations', 'Committee on Environment and Public Works']
102 110shrg41817 ['Committee on Appropriations', 'Committee on Health, Education, Labor, and Pensions']
105 110shrg38602 ['Committee on Appropriations', 'Committee on Energy and Natural Resources']
158 110shrg41620 ['Committee on Appropriations', 'Committee on Energy and Natural Resources']
164 110hhrg50138 ['Committee on Appropriations', 'Committee on Ways and Means']
170 110shrg34649 []
220 110hhrg37579 ['Committee on Energy and Commerce', 'Committee on Science and Technology']


In [11]:
# 108shrg91254: Committee on Environment and Public Works
# https://www.govinfo.gov/content/pkg/CHRG-108shrg91254/html/CHRG-108shrg91254.htm
# >> drop falsely assigned 'Committee on Appropriations'

# 110shrg41817: Committee on Health, Education, Labor, and Pensions
# https://www.govinfo.gov/content/pkg/CHRG-108shrg91254/html/CHRG-108shrg91254.htm
# >> drop falsely assigned 'Committee on Appropriations'

# 110shrg38602: Committee on Energy and Natural Resources
# https://www.govinfo.gov/content/pkg/CHRG-110shrg38602/html/CHRG-110shrg38602.htm
# >> drop falsely assigned 'Committee on Appropriations'

# 110shrg41620: Committee on Energy and Natural Resources
# https://www.govinfo.gov/content/pkg/CHRG-110shrg41620/html/CHRG-110shrg41620.htm
# >> drop falsely assigned 'Committee on Appropriations'

# 110hhrg50138: Committee on Ways and Means
# https://www.govinfo.gov/content/pkg/CHRG-110hhrg50138/html/CHRG-110hhrg50138.htm
# >> drop falsely assigned 'Committee on Appropriations'

# 110shrg34649: Committee on the Budget
# https://www.govinfo.gov/app/details/CHRG-110shrg34649/CHRG-110shrg34649
# >> fill in missing information

# 110hhrg37579: Committee on Energy and Commerce & Committee on Science and Technology
# https://www.govinfo.gov/content/pkg/CHRG-110hhrg37579/html/CHRG-110hhrg37579.htm
# >> This is the only hearing that was actually held in front of two committees. 
# >> To avoid pseudo-replication we assign this to the Committee on Energy and Commerce,
# >> which has taken the lead on this hearing (listed first on transcript & chairing).

In [12]:
# Make the corrections
for i, text in enumerate(t.texts):
    if text['identifier'] in ['108shrg91254', '110shrg41817', '110shrg38602', '110shrg41620', '110hhrg50138']:
        print(i, text['identifier'], text['committee'])
        t.texts[i]['committee'] = [t.texts[i]['committee'][1]]
        print(i, text['identifier'], text['committee'], '\n')

    if text['identifier'] in ['110shrg34649']:
        print(i, text['identifier'], text['committee'])
        t.texts[i]['committee'] = ['Committee on the Budget']
        print(i, text['identifier'], text['committee'], '\n')

    if text['identifier'] in ['110hhrg37579']:
        print(i, text['identifier'], text['committee'])
        t.texts[i]['committee'] = [t.texts[i]['committee'][0]]
        print(i, text['identifier'], text['committee'], '\n')      

25 108shrg91254 ['Committee on Appropriations', 'Committee on Environment and Public Works']
25 108shrg91254 ['Committee on Environment and Public Works'] 

102 110shrg41817 ['Committee on Appropriations', 'Committee on Health, Education, Labor, and Pensions']
102 110shrg41817 ['Committee on Health, Education, Labor, and Pensions'] 

105 110shrg38602 ['Committee on Appropriations', 'Committee on Energy and Natural Resources']
105 110shrg38602 ['Committee on Energy and Natural Resources'] 

158 110shrg41620 ['Committee on Appropriations', 'Committee on Energy and Natural Resources']
158 110shrg41620 ['Committee on Energy and Natural Resources'] 

164 110hhrg50138 ['Committee on Appropriations', 'Committee on Ways and Means']
164 110hhrg50138 ['Committee on Ways and Means'] 

170 110shrg34649 []
170 110shrg34649 ['Committee on the Budget'] 

220 110hhrg37579 ['Committee on Energy and Commerce', 'Committee on Science and Technology']
220 110hhrg37579 ['Committee on Energy and Commerce'] 


In [13]:
# Flatten the comittee lists
for i, text in enumerate(t.texts):
    t.texts[i]['committee'] = text['committee'][0]

<br>

## 4)  Create final hearings subset

In [14]:
# Drop hearing 111hhrg57926: hearing to review u.s. agriculture policy in advance of the 2012 farm bill 
# (This hearing has many industry (82) witnesses giving testimony, most of wich (77) represent the agribusiness and are mostly farmers. 
# This heavily skews the result, suggesting that the industry represented 58% as opposed to around 33% of testimonies in 2010, and misrepresents 
# the actual debate on climate legislation at the time. As the bill does not discuss climate legislation, it was excluded.)

# Drop hearings '111jhrg75408' and '111hhrg56971' as these were held by a commission and not by committees

t.texts = [text for text in t.texts if text['identifier'] not in 
           ['111hhrg57926', '111jhrg75408', '111hhrg56971']]
len(t)

260

In [15]:
# Drop hearings from Joint Economic Committee which do not fit into the analysis of the House vs the Senate (as they are both)
t.texts = [text for text in t.texts if text['committee'] != 
           'Joint Economic Committee']
len(t)

257

In [16]:
save_as(t, 'Hearings/06_witnesses.pkl') # Last completed on Mar 01, 2022